In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Load dataset
df = pd.read_csv("themeMate_dataset.csv")

# 2. Encode categorical columns
label_event = LabelEncoder()
label_theme = LabelEncoder()
label_location = LabelEncoder()

df["Event Type"] = label_event.fit_transform(df["Event Type"])
df["Theme"] = label_theme.fit_transform(df["Theme"])
df["Location"] = label_location.fit_transform(df["Location"])

# 3. Features
features = ["Event Type", "Theme", "Location", "Attendees"]

# 4. Cost Model
X_cost = df[features]
y_cost = df["Estimated Cost (INR)"]
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_cost, y_cost, test_size=0.2, random_state=1)
cost_model = LinearRegression()
cost_model.fit(X_train_c, y_train_c)
rmse_cost = np.sqrt(mean_squared_error(y_test_c, cost_model.predict(X_test_c)))

# 5. Success Score Model
y_success = df["Predicted Success Score"]
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_cost, y_success, test_size=0.2, random_state=1)
success_model = LinearRegression()
success_model.fit(X_train_s, y_train_s)

# 6. Prediction & Recommendation Function
def advisor(event_type, theme, location, attendees, budget):
    try:
        e = label_event.transform([event_type])[0]
        t = label_theme.transform([theme])[0]
        l = label_location.transform([location])[0]
    except ValueError:
        return {"error": "Invalid input. Please check the event type, theme, or location."}

    X_input = [[e, t, l, attendees]]

    # Cost prediction
    cost = cost_model.predict(X_input)[0]
    lower = int(cost - rmse_cost)
    upper = int(cost + rmse_cost)

    # Success score prediction
    success = success_model.predict(X_input)[0]

  # Suggestions under budget and same Event Type
    suggestions = df[
        (df["Estimated Cost (INR)"] <= budget) &
        (df["Attendees"] >= attendees - 50) &
        (df["Event Type"] == e)  # Only same event type
    ].sort_values(by="Estimated Cost (INR)").head(3)

    # Decode suggestions
    suggestions["Event Type"] = label_event.inverse_transform(suggestions["Event Type"])
    suggestions["Theme"] = label_theme.inverse_transform(suggestions["Theme"])
    suggestions["Location"] = label_location.inverse_transform(suggestions["Location"])


    return {
        "estimated_cost": int(cost),
        "confidence_range": (lower, upper),
        "success_score": round(float(success), 2),
        "suggestions": suggestions[["Event Type", "Theme", "Location", "Estimated Cost (INR)"]].to_dict(orient="records")
    }

# 7. User Interface
print("🎯 ThemeMate AI – Smart Event Planner")
print("\nAvailable Event Types:", list(label_event.classes_))
print("Available Themes (sample):", list(label_theme.classes_)[:5], "...")
print("Available Locations:", list(label_location.classes_))

event = input("\nEnter Event Type: ").strip().title()
theme = input("Enter Theme: ").strip().title()
location = input("Enter Location: ").strip().title()
attendees = int(input("Enter Number of Attendees: "))
budget = int(input("Enter Your Maximum Budget (₹): "))

result = advisor(event, theme, location, attendees, budget)

# 8. Output
if "error" in result:
    print("\n❌", result["error"])
else:
    print(f"\n💸 Estimated Cost: ₹{result['estimated_cost']} (± ₹{int(rmse_cost)})")
    print(f"📈 Predicted Success Score: {result['success_score']} / 1.0")
    print(f"📊 Confidence Range: ₹{result['confidence_range'][0]} – ₹{result['confidence_range'][1]}")

    if result["suggestions"]:
        print("\n💡 Budget-Friendly Suggestions:")
        for s in result["suggestions"]:
            print(f"→ {s['Event Type']} | {s['Theme']} | {s['Location']} | ₹{int(s['Estimated Cost (INR)'])}")
    else:
        print("\n⚠️ No alternatives found under your budget.")


🎯 ThemeMate AI – Smart Event Planner

Available Event Types: ['Birthday', 'Corporate', 'Cultural', 'Sports', 'Wedding']
Available Themes (sample): ['Blue & White', 'Champion’s Podium', 'Ethnic Vibe', 'Festive Lights', 'Garden Dream'] ...
Available Locations: ['Bangalore', 'Chennai', 'Delhi', 'Hyderabad', 'Mumbai']

💸 Estimated Cost: ₹102696 (± ₹52923)
📈 Predicted Success Score: 0.82 / 1.0
📊 Confidence Range: ₹49773 – ₹155619

💡 Budget-Friendly Suggestions:
→ Birthday | Retro Party | Delhi | ₹23584
→ Birthday | Unicorn Fantasy | Mumbai | ₹25514
→ Birthday | Unicorn Fantasy | Hyderabad | ₹26402


c:\Users\B123s\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
c:\Users\B123s\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
